In [1]:
import decisiontree_gpu as dtree
import numpy as np
import time

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("DTREE-SUSY").config("spark.executor.heartbeat", "3600s").getOrCreate()
sc = spark.sparkContext


In [2]:
# 1. Leemos el dataset 
rdd = sc.textFile('../datasets/SUSY.csv', 225)
def append_ret(a,b):
    a.append(b)
    return a
# 2. Separamos y ponemos la etiqueta al final en vez de al principio
rdd = rdd.map(lambda line: line.split(','))
rdd = rdd.map(lambda line: append_ret(line[1:], line[0]))


MapPartitionsRDD[6] at coalesce at NativeMethodAccessorImpl.java:0

In [3]:
rdd.getNumPartitions()
    

In [77]:
def gpu_work(max_depth=6, min_samples_per_node=1, tpb=128):
    def _gpu_work(data):
        # 1. Procesamos el dataset
        inp = np.asarray(list(data), dtype=np.float32)
        return [dtree.train_tree(inp, max_depth, min_samples_per_node, tpb)]    
    return _gpu_work

In [78]:
def predict(sample, trees):
    counter = 0
    for tree in trees:
        counter += dtree.predict(sample, tree)
    
    return 1 if counter > len(trees) / 2 else 0

def evaluar(dataset, trees):
    aciertos = 0
    for sample in dataset:
        etiqueta = predict(sample, trees)
        if int(etiqueta) == int(sample[-1]):
            aciertos += 1
        
    return aciertos / dataset.shape[0]


In [79]:
# El número de árboles seleccionado se corresponde a evaluar 20000 muestras por árbol.
# Realizmos validación cruzada para profundidad 6 y 7 en nuestro modelo y en Spark

def eval_model_gpu(rdd, ntrees, max_depth, seed):
    # Ejecutamos el modelo sobre las particiones [Profundidad 6 Iteración 0]
    # Subdividimos el RDD en Split de Training (90 %) y Split de Test (10 %)
    (trainingData, testData) = rdd.randomSplit([0.9, 0.1], seed=seed)
    # 4. Damos el número de particiones deseadas en el RDD para trabajar con el modelo
    trainingData = trainingData.repartition(ntrees)
    inicio = time.time()
    arboles = trainingData.mapPartitions(gpu_work(max_depth=max_depth))
    arboles = arboles.collect()
    fin = time.time()
    precision = evaluar(np.float32(testData.collect()), arboles) * 100
    print('Tiempo utilizado:', fin-inicio)
    print('Precisión', precision)

In [21]:
eval_model_gpu(rdd, 225, 6, 0)

Tiempo utilizado: 560.4573318958282
Precisión 78.11074853556067


In [22]:
eval_model_gpu(rdd, 225, 6, 1)

Tiempo utilizado: 536.0125508308411
Precisión 78.13558916148808


In [17]:
eval_model_gpu(rdd, 225, 6, 2)

Tiempo utilizado: 519.9591224193573
Precisión 78.24470219097861


In [18]:
eval_model_gpu(rdd, 225, 6, 3)

Tiempo utilizado: 519.4733338356018
Precisión 78.14667206836474


In [19]:
eval_model_gpu(rdd, 225, 6, 4)

Tiempo utilizado: 530.5869901180267
Precisión 78.12771359958798


In [20]:
eval_model_gpu(rdd, 225, 6, 5)

Tiempo utilizado: 543.3472962379456
Precisión 78.1055959506849


In [23]:
eval_model_gpu(rdd, 225, 6, 6)

Tiempo utilizado: 545.5358641147614
Precisión 78.14674603919002


In [80]:
eval_model_gpu(rdd, 225, 6, 7)

Tiempo utilizado: 530.7353973388672
Precisión 78.10162888034186


In [81]:
eval_model_gpu(rdd, 225, 6, 8)

Tiempo utilizado: 558.4582099914551
Precisión 78.17898363548532


In [82]:
eval_model_gpu(rdd, 225, 6, 9)

Tiempo utilizado: 521.2870435714722
Precisión 78.10271141036057


In [83]:
eval_model_gpu(rdd, 225, 7, 0)

Tiempo utilizado: 583.1255974769592
Precisión 78.84102226346936


In [84]:
eval_model_gpu(rdd, 225, 7, 1)

Tiempo utilizado: 559.2700748443604
Precisión 78.82926031027615


In [85]:
eval_model_gpu(rdd, 225, 7, 2)

Tiempo utilizado: 584.0724532604218
Precisión 78.96584692091359


In [86]:
eval_model_gpu(rdd, 225, 7, 3)

Tiempo utilizado: 548.7389233112335
Precisión 78.85953655825352


In [87]:
eval_model_gpu(rdd, 225, 7, 4)

Tiempo utilizado: 561.0129799842834
Precisión 78.78107352245438


In [88]:
eval_model_gpu(rdd, 225, 7, 5)

Tiempo utilizado: 574.2824943065643
Precisión 78.76424834788024


In [89]:
eval_model_gpu(rdd, 225, 7, 6)

Tiempo utilizado: 572.0794265270233
Precisión 78.7879392612608


In [90]:
eval_model_gpu(rdd, 225, 7, 7)

Tiempo utilizado: 580.5854170322418
Precisión 78.83852708180727


In [91]:
eval_model_gpu(rdd, 225, 7, 8)

Tiempo utilizado: 576.3165907859802
Precisión 78.80737461124797


In [92]:
eval_model_gpu(rdd, 225, 7, 9)

Tiempo utilizado: 592.1349990367889
Precisión 78.76863682910908


In [3]:
"""
Versión de Spark, se espera que sea más precisa, pero mucho más lenta.
"""
import time
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.linalg import Vectors
from pyspark.sql import Row
from pyspark.ml.evaluation import BinaryClassificationEvaluator

rdd2= sc.textFile('../datasets/SUSY.csv')
rdd2 = rdd2.map(lambda line: line.split(','))
def append_ret(a,b):
    a.append(b)
    return a
rdd2 = rdd2.map(lambda line: append_ret(line[1:], line[0]))

rdd2 = rdd2.map(lambda x: Row(label=float(x[-1]), features=Vectors.dense(x[:-2])))
df = rdd2.toDF()

def eval_spark(ntrees=225, max_depth=6, seed=0):   
    inicio = time.time()
    (train, test) = df.randomSplit([0.9,0.1], seed=seed)
    rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label', numTrees=ntrees, 
                            featureSubsetStrategy='all', subsamplingRate=1/ntrees,
                            maxDepth=max_depth, seed=seed)
    rfModel = rf.fit(train)
    fin = time.time()
    result = rfModel.transform(test)
    evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")
    accuracy = evaluator.evaluate(rfModel.transform(test))
    
    print('Tiempo utilizado:', fin - inicio)
    print('Precisión', accuracy * 100)

In [4]:
eval_spark(225, 6, 0)

Tiempo utilizado: 1201.0418632030487
Precisión 85.29685357004023


In [5]:
eval_spark(225, 6, 1)

Tiempo utilizado: 1273.03622174263
Precisión 85.30343555609613


In [6]:
eval_spark(225, 6, 2)

Tiempo utilizado: 1236.4733180999756
Precisión 85.37577721342872


In [7]:
eval_spark(225, 6, 3)

Tiempo utilizado: 1238.7885835170746
Precisión 85.32967269446206


In [8]:
eval_spark(225, 6, 4)

Tiempo utilizado: 1334.7991597652435
Precisión 85.33446170864863


In [9]:
eval_spark(225, 6, 5)

Tiempo utilizado: 1324.342830657959
Precisión 85.24477843225576


In [10]:
eval_spark(225, 6, 6)

Tiempo utilizado: 1187.4351456165314
Precisión 85.27251217304988


In [11]:
eval_spark(225, 6, 7)

Tiempo utilizado: 1247.093768119812
Precisión 85.28868682446948


In [12]:
eval_spark(225, 6, 8)


Tiempo utilizado: 1250.9972622394562
Precisión 85.33649649957039


In [14]:
eval_spark(225, 6, 9)



Tiempo utilizado: 1268.3399126529694
Precisión 85.27754012654106


In [15]:
eval_spark(225, 7, 0)

Tiempo utilizado: 1280.4425477981567
Precisión 85.29685357004017


In [ ]:
eval_spark(225, 7, 1)

In [ ]:
eval_spark(225, 7, 2)

In [ ]:
eval_spark(225, 7, 3)

In [ ]:
eval_spark(225, 7, 4)

In [ ]:
eval_spark(225, 7, 5)

In [ ]:
eval_spark(225, 7, 6)

In [ ]:
eval_spark(225, 7, 7)

In [ ]:
eval_spark(225, 7, 8)

In [ ]:
eval_spark(225, 7, 9)